In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-09 03:02:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-02-09 03:02:17 (6.08 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Sample_FR").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_fr.tsv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sample_fr.tsv"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")
# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...| 2013-02-13|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game of Thrones...|Digital_Ebook_Pur

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

49
49
49


In [ ]:
# Transform the dataset to fit the tables in the schema file.
# Be sure the DataFrames match in data type and in column name.
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



**Create Tables**

In [ ]:
# Create review_id_table 
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2I1383CZ4QBZZ|      28465|B00004VXU0|     735114749| 2015-04-13|
|R1L9V8TG9GSZJ2|      29042|B008SOJQVE|     244062959| 2013-11-25|
|R1I34RKB2COZ8S|      32799|B008YISMLY|     494115014| 2013-03-31|
|R35CGHFIHGVQQM|      26552|B00D1VKZ3M|      16141818| 2015-01-22|
| RSOWF1UJJVMVI|      28700|B00I7VMIC2|     551407899| 2014-07-06|
|R20RF32ZCTMRR3|      32494|B00CPZ7DBW|     788907793| 2014-06-16|
|R2ZNPOPXQWHYYF|      29019|0316225932|     719455430| 2015-04-28|
|R2Y4O06QMOGHD0|      24196|2226249672|     753674225| 2013-10-03|
|R2E7QEWSC6EWFA|      18940|B00CW7KK9K|     977480037| 2014-06-16|
|R26E6I47GQRYKR|      20315|B002L6SKIK|     827187473| 2013-06-10|
|R2VGNSD09UEW5T|      25935|B00XO12REY|     271191040| 2015-07-21|
|R1BSSQ563EWKNN|      28692|0006486118|     586496321| 2013-09

In [ ]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Create products table 
products = df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00004VXU0|    The Big Lebowski|
|B008SOJQVE|      The Connection|
|B008YISMLY| Peter Pan [Blu-ray]|
|B00D1VKZ3M|Bridget Jones: Ma...|
|B00I7VMIC2|True Detective - ...|
|B00CPZ7DBW|     Jump Birdy Jump|
|0316225932|    The Burning Room|
|2226249672|Au revoir là-haut...|
|B00CW7KK9K|Withings Pulse - ...|
|B002L6SKIK|          Prometheus|
|B00XO12REY|  The Monsanto Years|
|0006486118|A Song of Ice and...|
|B000FUM0TE|             Amistad|
|B00B0ID7D8|LEGO Calendriers ...|
|B0044JV1K6|Braddock - Missin...|
|B00005UMYC|           O'Brother|
|B00170MYIS|Sony NP-FG1 Batte...|
|B00004YLIU|               Kid A|
|B004GJXQ20|A Game of Thrones...|
|B00CSR2J9I|   Hill Climb Racing|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create customer table
from pyspark.sql.functions import desc
customer_count_df = df.select("customer_id").groupby("customer_id")\
  .agg({"customer_id":"count"})
customer_count_df = customer_count_df.withColumnRenamed("count(customer_id)", "customer_count")
customer_count_df.show(truncate=False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|26425      |1             |
|29814      |1             |
|20913      |1             |
|28767      |1             |
|20842      |1             |
|31630      |1             |
|31660      |1             |
|17564      |1             |
|24207      |2             |
|32494      |1             |
|14952      |2             |
|24196      |5             |
|28091      |3             |
|28448      |1             |
|31588      |1             |
|28700      |1             |
|32799      |3             |
|28557      |1             |
|28692      |1             |
|21490      |1             |
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create vine table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2I1383CZ4QBZZ|          3|            1|          4|   N|
|R1L9V8TG9GSZJ2|          5|            0|          0|   N|
|R1I34RKB2COZ8S|          5|            0|          0|   N|
|R35CGHFIHGVQQM|          2|            2|          2|   N|
| RSOWF1UJJVMVI|          5|            1|          1|   N|
|R20RF32ZCTMRR3|          5|            2|          2|   N|
|R2ZNPOPXQWHYYF|          4|            0|          0|   N|
|R2Y4O06QMOGHD0|          5|            2|          3|   N|
|R2E7QEWSC6EWFA|          4|            0|          1|   N|
|R26E6I47GQRYKR|          2|            3|          5|   N|
|R2VGNSD09UEW5T|          1|            1|          9|   N|
|R1BSSQ563EWKNN|          5|            0|          0|   N|
|R3UYE0U7SQCI8Q|          5|            2|          3|   N|
|R3L6J8KA59DRLS|          2|            

Postgres Setup

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<end point>:5432/sample_fr"
config = {"user":"<user>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS

customer_count_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)